# Try this with Dask First

I'm going to try this firstly with Dask and then with Spark. I'm more confident in Dask than I am in Spark. It will also let us confirm our answer - triangulate the data so to speak.

In [85]:
#just finding my exact file name here...
!ls

03_test.ipynb	       fhvhv
04_pyspark.ipynb       fhvhv_tripdata_2021-01.csv
05_taxi_schema.ipynb   fhvhv_tripdata_2021-01.csv.1
06_spark_sql.ipynb     fhvhv_tripdata_2021-02.csv
07_groupby_join.ipynb  fhvhv_tripdata_2021-02_homeworkQ3.ipynb
08_rdds.ipynb	       head.csv
data		       HomeworkQ3.ipynb
download_data.sh       test.py


In [90]:
import dask.dataframe as dd

In [91]:
filename = 'fhvhv_tripdata_2021-02.csv'
df = dd.read_csv(filename, parse_dates=['pickup_datetime', 'dropoff_datetime'])

In [92]:
df.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [93]:
# create a new dataframe with only pickup_datetime on the 15h of February
fifteen_Feb_df = df[(df.pickup_datetime >= '2021-02-15') &
                    (df.pickup_datetime < '2021-02-16') ]

In [94]:
my_15_feb_df = fifteen_Feb_df.compute() #turn dask dataframe it into a pandas dataframe

# Answer in Dask

In [95]:
my_15_feb_df.shape

(367170, 7)

In [53]:
#just having a quick look at the data here...
my_15_feb_df.head(10)

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
151549,HV0003,B02764,2021-02-15 00:14:30,2021-02-15 00:26:11,108,22,NaN
151550,HV0005,B02510,2021-02-15 00:23:46,2021-02-15 00:41:09,37,61,NaN
151551,HV0004,B02800,2021-02-15 00:34:20,2021-02-15 00:39:34,82,82,NaN
151552,HV0004,B02800,2021-02-15 00:49:51,2021-02-15 01:10:29,138,239,NaN
151553,HV0003,B02884,2021-02-15 00:03:13,2021-02-15 00:19:23,81,47,NaN
151554,HV0003,B02884,2021-02-15 00:33:30,2021-02-15 00:46:24,69,243,NaN
151555,HV0003,B02884,2021-02-15 00:49:01,2021-02-15 00:54:45,243,243,NaN
151556,HV0003,B02884,2021-02-15 00:55:56,2021-02-15 01:04:04,127,116,NaN
151557,HV0005,B02510,2021-02-15 00:03:57,2021-02-15 00:13:38,81,240,NaN
151558,HV0005,B02510,2021-02-15 00:39:03,2021-02-15 00:46:41,259,259,NaN


In [54]:
my_15_feb_df.tail(10)

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
518709,HV0003,B02866,2021-02-15 23:56:17,2021-02-16 00:09:05,186,146,NaN
518710,HV0003,B02866,2021-02-15 23:06:59,2021-02-15 23:21:53,148,143,NaN
518711,HV0003,B02866,2021-02-15 23:56:21,2021-02-16 00:04:42,79,125,NaN
518712,HV0003,B02875,2021-02-15 23:02:04,2021-02-15 23:17:12,238,237,NaN
518713,HV0003,B02875,2021-02-15 23:22:51,2021-02-15 23:28:15,237,233,NaN
518714,HV0003,B02875,2021-02-15 23:38:19,2021-02-15 23:48:02,170,88,NaN
518715,HV0003,B02765,2021-02-15 23:30:45,2021-02-16 00:02:46,65,91,NaN
518716,HV0003,B02764,2021-02-15 23:16:13,2021-02-16 00:03:08,155,265,NaN
518717,HV0003,B02764,2021-02-15 23:24:45,2021-02-15 23:51:05,225,92,NaN
518718,HV0003,B02682,2021-02-15 23:22:15,2021-02-16 00:01:52,217,217,NaN


# Now try with pyspark

In [55]:
import pyspark
from pyspark.sql import SparkSession

In [56]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('fhvhv_tripdata_2021-02') \
    .getOrCreate()

22/02/27 14:08:45 WARN Utils: Your hostname, sandy-ODYSSEY-X86J4105 resolves to a loopback address: 127.0.1.1; using 10.0.0.1 instead (on interface wlo2)
22/02/27 14:08:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/27 14:08:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [57]:
#!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

In [58]:
#!wc -l fhvhv_tripdata_2021-02.csv

In [59]:
from pyspark.sql import types

In [60]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

I don't need to bother repartitioning etc. here as I've already done it before. I'll turn these code cells into markdown cells and simply start off from reading the parquet file

df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

df = df.repartition(24)

df.write.parquet('fhvhv/2021/02/')

In [68]:
df = spark.read.parquet('fhvhv/2021/02/')

In [69]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



# Answer in Pyspark

In [70]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("fhvhv_2021_02")

sqlDF = spark.sql("SELECT COUNT(1) \
                     FROM fhvhv_2021_02 \
                    WHERE pickup_datetime >= '2021-02-15' AND pickup_datetime < '2021-02-16'")
sqlDF.show()

+--------+
|count(1)|
+--------+
|  367165|
+--------+

